In [ ]:
# adds the modules from this repo to the path
push!(LOAD_PATH, "../")

In [ ]:
# external dependencies
using Revise
using QuantumClifford
using LaTeXStrings
using ProgressMeter
using Utilities
using Plots
Plots.pyplot()
using Nemo
using JLD2

In [ ]:
# modules in this repo
using FieldConversions
using Automata
using CircuitLabels
using PlotUtils
using AutomataPlots

In [ ]:
default(titlefont = ("times"), framestyle = :box, grid=false,linewidth=1.5,tickfontsize=9,legendfontsize=8.5,
    labelfontsize=12)

In [ ]:
automata = read_automata([(time = 2, space = 2, shift = 0), (time = 1, space = 2,shift=1), 
        (time = 4, space = 4, shift = 0)]; dir = "../data/");

### Square lattice plots

  - Index 8: non-fractal good scrambling class.
  - Index 9: fractal d_f=1.9 class.
  - Index 15: SDKI class.

In [ ]:
titles = Dict(8=>"Nonfractal good scrambling class", 15=>"SDKI class", 9=>"Fractal " * L"d_f\approx 1.9" * " class");

In [ ]:
rep_idxs = [get_idx_1d(idx) for idx=[[1,1],[1,2],[1,3],[3,3],[2,3],[2,2]]];

In [ ]:
[get_gate_label(idx) for idx in rep_idxs]

### Integer time steps

In [ ]:
tf=256
dims=(time=2,space=2,shift=0)
automaton_powers = Dict()
@showprogress for idx in [8,9,15]
    pows = get_powers(automata[dims][idx]; tf=tf)
    automaton_powers[idx] = [automaton_to_matrix(Matrix(pows[t+1]), t) for t=0:tf]
end

In [ ]:
stabs = Dict()
# if running on laptop, best to only try plotting to shortish time
tf=64
for idx in [8,9,15]
    stabs[idx] = [get_pauli_image(automaton_powers[idx],i,1; tf=tf, stabilize=false) 
        for i=1:2*dims.space]
end

In [ ]:
all_plots = []
bins = Bool[0,0,1,0]
tag="Z"
for idx=[8,9,15]
    tag, p = plot_together(bins, stabs[idx], dims.space; print = true, aspect = dims.time)
    push!(all_plots, p)
end
plt = Plots.plot(all_plots..., layout=(1,3),size=(900,300),axis=false)

In [ ]:
# Plot operator spreading for each Pauli within one unit cell
idx = 9
all_plots = []
for i=1:2^(2*dims.space)-1
    bins = get_binaries(i, 2*dims.space)
    tag, p = plot_together(bins, stabs[idx], dims.space, aspect = dims.space)
    push!(all_plots, Plots.plot(p, title = tag, xlabel = L"x", ylabel = L"t"))
end
plt = Plots.plot(all_plots...)
p = plot_suptitle(plt, "Operator spreading in iSWAP-core square-lattice circuit, " * L"(v_+, v_-) = " *get_gate_label(idx), 
    num_plots = length(all_plots), rel_size = (1,0.7))

In [ ]:
# plot image on odd and even sites separately
idx = 15
bins = Bool[0, 0, 1,0]
tag, all_plots = plot_separate(bins, stabs[idx], dims.space; print=true)
for j=1:dims.space
    if j%2==0
        Plots.plot!(all_plots[j],yformatter=_->"")
    else
        Plots.plot!(all_plots[j], ylabel = L"t")
    end
    Plots.plot!(all_plots[j], xlabel = L"n")
    Plots.annotate!((-tf*0.85, tf/4,get_legend_key(j, lab="j")), annotationhalign=:left,annotationfontsize=12)
end
plt=Plots.plot(all_plots...,layout=(1,2),share_y=true)
p = plot_suptitle(plt, "$(titles[idx]), $(tag)", size = (600,300))

In [ ]:
# plot I, X, Y, Z in footprint separately
idx = 8
tag, plots = plot_paulis(bins, stabs[idx], dims.space, dims.time; print=true)
plt=Plots.plot(plots...,layout=(2,2),share_x=true, share_y=true)
p = plot_suptitle(plt, "$(titles[idx]), $(tag)",size = (550,450))

In [ ]:
### Run out to much longer times and fit fractal dimension
idx = 9
tf = 1024
pows = get_powers(automata[dims][idx]; tf=tf)
automat_pows = [automaton_to_matrix(Matrix(pows[t+1]), t) for t=0:tf]
stabs_long = [get_pauli_image(automat_pows,i,dims.time÷dims.space; tf=tf, stabilize=false) 
        for i=1:2*dims.space]

In [ ]:
all_plots = []
res_plots = []
legend=:topleft
all_fits = []
all_counts = []
@showprogress for i=1:2^(2*dims.space)-1
    tag, plots, fits, counts = fit_paulis(get_binaries(i, 2*dims.space), stabs_long, dims.space, dims.time; print=true, xmin=2^6)
    push!(all_plots, Plots.plot(plots[1], title = tag, legend = legend))
    push!(res_plots, Plots.plot(plots[2], title = tag, legend = legend))
    push!(all_fits, fits)
    push!(all_counts, counts)
    legend = false
end

In [ ]:
xdata = 1:2^(2*dims.space)-1
plt_fits = Plots.plot(xlabel = L"i", ylabel = L"d_f", title = "Fits", legend=false)
for (pauli_i,pauli) in enumerate([L"I", L"X", L"Y", L"Z"])
    use_x = filter(x->haskey(all_fits[x], pauli), xdata)
    Plots.scatter!(use_x, [all_fits[i][pauli][1][1] for i=use_x], yerror=[all_fits[i][pauli][2][1] for i=use_x],
        msc=AutomataPlots.COLOR_ARR[pauli_i], color = AutomataPlots.COLOR_ARR[pauli_i])
end
plt = Plots.plot(all_plots..., plt_fits)
p = plot_suptitle(plt, "Cumulative counts of Paulis within light cone, $(titles[idx])", num_plots = length(all_plots))

In [ ]:
plt = Plots.plot([Plots.plot(p, xlabel = L"t", ylabel = L"\log \sum N_\sigma (t')-" * "fit", xscale=:log2) 
        for p in res_plots]..., plt_fits)
p = plot_suptitle(plt, "Residuals from fit to fractal dimension, $(titles[idx])", num_plots=length(res_plots))

### Tracking the trace

In [ ]:
tf=2^14;

In [ ]:
trace_pows = get_automaton_traces(automata[dims][rep_idxs]; tf=tf);

In [ ]:
laurent_arrs = [[laurent_to_array(trace[t], t, check=false) for t=1:tf]
    for trace in trace_pows];

In [ ]:
trace_plots = Dict()
for idx_i=1:length(rep_idxs)
    trace_plots[rep_idxs[idx_i]] = plot_trace(laurent_arrs[idx_i], 2^8; ttl = get_gate_label(rep_idxs[idx_i]), xmin=2^6)
end

In [ ]:
plt= Plots.plot([trace_plots[idx][:plot_nonzero] for idx in rep_idxs]..., titlefontsize=10)
p = plot_suptitle(plt, "Traces of CNOT CQCA, number of nonzero coefficients", num_plots = length(rep_idxs))

In [ ]:
plt = Plots.plot([trace_plots[idx][:plot_2d] for idx in rep_idxs]..., titlefontsize=10)
p = plot_suptitle(plt, "Traces of iSWAP-core CQCA", num_plots = length(rep_idxs),
    rel_size = (1.5,1))

In [ ]:
all_plots = []
for idx in rep_idxs
    if haskey(trace_plots[idx], :fit_plots) && trace_plots[idx][:fits][1]>1.05
        push!(all_plots, [Plots.plot(p, titlefontsize=10) for p in trace_plots[idx][:fit_plots]]...)
    end
end
plt, use_size = plot_layout(all_plots, 2; force_grid=true)
p = plot_suptitle(plt, "Traces of iSWAP-core CQCA, Fits to fractal dimension", size=use_size)

### Half-integer time steps

In [ ]:
dims = (time = 1, space = 2, shift = 1)
shift_mat = get_shift_mat(dims.space, dims.shift);
inv_shift = inv(shift_mat);

In [ ]:
shift_powers = Dict()
tf=512
@showprogress for idx=[8,9,15]
    pows = get_powers(shift_mat * automata[dims][idx]; tf=tf)
    # "undo" the shift to recenter the automaton
    shift_powers[idx] = [automaton_to_matrix(Matrix(inv_shift^(t) * pows[t+1]), 2*t) for t=0:tf]
end

In [ ]:
stabs_shift = Dict()
tf=128
for idx=keys(shift_powers)
    stabs_shift[idx] = [get_pauli_image(shift_powers[idx],i,dims.time/dims.space, tf=tf, stabilize=false) 
        for i=1:2*dims.space]
    for i=1:2*dims.space # matches (time = 2, space = 2, shift = 0) at integer time steps
        @assert stabs_shift[idx][i][1:2:end,:]==stabs[idx][i]
    end
end

In [ ]:
bins = Bool[0,0,1,0]
tag="Z"
all_plots = []
for idx=[8,9,15]
    tag, p = plot_together(bins, stabs_shift[idx], dims.space)
    push!(all_plots, Plots.plot(p,aspect_ratio=1))
end
plt = Plots.plot(all_plots..., layout=(1,3),size=(900,300),axis=false)

In [ ]:
periods = get_automaton_periods([shift_mat * automata[dims][idx] for idx=1:length(automata[dims])], [16:2:256;], dims, rep_idxs; 
    max_t=1000, save = false);

In [ ]:
dims

In [ ]:
periods[2]

In [ ]:
recurrence_times = load("../data/periods/$(dims.time)x$(dims.space).jld2");

In [ ]:
lengths = 16:2:256;
period_arrs = [[recurrence_times["periods"][idx][L] for L in lengths] for idx in rep_idxs];

In [ ]:
p = Plots.plot(xlabel = L"m", ylabel = L"\tau(m)", legend=:topleft,
    title = "Periods of poor-scrambling iSWAP-core Circuits",
    titlefontsize=10)
for idx_i=1:length(rep_idxs)
    if any(t->t>recurrence_times["tmax"], period_arrs[idx_i])
        continue
    end
    @assert period_arrs[idx_i]==[periods[rep_idxs[idx_i]][L] for L in lengths]
    Plots.plot!(lengths ./ dims.space, period_arrs[idx_i]./2, label=get_gate_label(rep_idxs[idx_i]))
end
for mult=1:3
    Plots.plot!(lengths ./ dims.space, mult * lengths ./ dims.space, color=:black, label="",linestyle=:dash)
end
p

### Kagome lattice

In [ ]:
dims = (time = 4, space = 4, shift = 0);

In [ ]:
idxs = [2,3];

In [ ]:
R, x = LaurentPolynomialRing(GF(2), "x");

In [ ]:
kagome_powers = Dict()
tf=128
num = 3
for idx=idxs
    pows = get_powers(automata[dims][idx]; tf=tf)
    kagome_powers[idx] = [[automaton_to_matrix(x^j * pows[t+1], t+j) for t=0:tf] for j=0:num-1]
end

In [ ]:
kagome_stabs = Dict()
for idx=idxs
    kagome_stabs[idx] = [[get_pauli_image(kagome_powers[idx][j],i,dims.time÷dims.space, 
            tf=tf, stabilize=false,padding = num) 
        for i=1:2*dims.space] for j=1:num]
end

In [ ]:
idx = 2
all_plots = []
for paulis in [[QuantumClifford.P"XXXX", QuantumClifford.P"IIII"],[QuantumClifford.P"XXXX", QuantumClifford.P"XXXX"]]
    pauli_ops = [stab_to_gf2(Stabilizer([pauli]))[1,:] for pauli in paulis]
    tag, p, _ = plot_multicell(pauli_ops, kagome_stabs[idx][[1,3]], dims.space; aspect = dims.space)
    push!(all_plots, Plots.plot(p, title = tag))
end
p = plot_suptitle(Plots.plot(all_plots...), L"D_6" * "-symmetric kagome circuit", size = (600, 300))

In [ ]:
idx = 3
tag, all_plots = plot_separate(Bool[0,0,0,0,1,0,0,0], kagome_stabs[idx][1] .+ kagome_stabs[idx][2], dims.space)
for j=1:dims.space
    if j%2==0
        Plots.plot!(all_plots[j],yformatter=_->"")
    else
        Plots.plot!(all_plots[j], ylabel = L"t/T")
    end
    Plots.plot!(all_plots[j], xlabel = L"n")
    Plots.annotate!((-tf*0.85, tf/4,get_legend_key(j, lab="j")), annotationhalign=:left,annotationfontsize=12)
end
plt=Plots.plot(all_plots...,layout=(2,2),share_x=true, share_y=true)
p = plot_suptitle(plt, "CNOT class kagome lattice circuit", size=(600,500))

In [ ]:
tf=64;

In [ ]:
### Compare to CNOT model
automat = R[x+1 x 0 0; 1 1 0 0; 0 0 1 1; 0 0 x^(-1) 1 + x^(-1)];

In [ ]:
Ry, y = PolynomialRing(R, "y");

In [ ]:
cnot_pows=get_traces(automat,charpoly(Ry, automat); tf=tf);

In [ ]:
laurent_cnot = [laurent_to_array(cnot_pows[t], size(automat,2) * t, check=true) for t=1:tf];

In [ ]:
d6_pows = get_traces(automata[dims][2], charpoly(Ry, automata[dims][2]); tf=tf);

In [ ]:
laurent_d6 = [laurent_to_array(d6_pows[t], size(automat,2) * t, check=true) for t=1:tf];

In [ ]:
p = Plots.plot(aspect_ratio=:equal, legend=false, xlims=(-tf-tf/50,tf+tf/50), ylims=(-tf/50,tf+tf/50),
    size = (600,300), xlabel = L"n", ylabel = L"t")
sz = 1.5
for t=1:tf
    if t%3==0
        Plots.scatter!(laurent_cnot[t], fill(t, length(laurent_cnot[t])), color=:red, markersize=sz, msc=:auto)
        @assert isempty(laurent_d6[t])
    else
        @assert laurent_d6[t]==laurent_cnot[t]
    end
    Plots.scatter!(laurent_d6[t], fill(t, length(laurent_d6[t])), color=:black, markersize=sz, msc=:auto)
end
p

In [ ]:
versioninfo()